In [23]:
import h5py
import numpy as np
import nrrd
import os
from helpers import *

def read_hdf5_file(file_path):
    # Open the HDF5 file
    with h5py.File(file_path, 'r') as hdf5_file:
        # Print all dataset names
        print("Datasets in the file:")
        for name in hdf5_file:
            print(name)
        
        # Read datasets into NumPy arrays
        data = {}
        for name in hdf5_file:
            data[name] = np.array(hdf5_file[name])
            print(f"{name} shape: {data[name].shape}")

    return data

In [24]:
current_directory = os.getcwd()
base_pred_path = f'{current_directory}/data/Vesuvius/test/predictions/'
filename = '3350_4000_8450_xyz_256_res1_s4'
filename = 'layers_1'
pred_path = f'{base_pred_path}{filename}_predictions.h5'
pred_data = read_hdf5_file(pred_path)
gt_label_data = read_hdf5_file(f'{current_directory}/data/Vesuvius/train/dataset/{filename}.h5') #may need to remove/add _raw if using layers_X or not

img_path = f'{current_directory}/data/Vesuvius/test/raw/{filename}.nrrd'
img, _ = nrrd.read(img_path)

pred = pred_data['predictions']
gt = gt_label_data['label']
print(pred.shape, img.shape, np.sum(pred), np.sum(gt))

Datasets in the file:
predictions
predictions shape: (256, 256, 256)
Datasets in the file:
label
raw
label shape: (256, 256, 256)
raw shape: (256, 256, 256)
(256, 256, 256) (256, 256, 256) 6675351 6480098


In [25]:
#show prediction
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(pred[:,slice_index,:])
    elif axis == 2:
        plt.imshow(pred[:,:,slice_index])
    else:
        plt.imshow(pred[slice_index,:,:])
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

interact(plot_slice, slice_index=IntSlider(min=0, max=pred.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [26]:
#show ground truth label
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(gt[:,slice_index,:])
    elif axis == 2:
        plt.imshow(gt[:,:,slice_index])
    else:
        plt.imshow(gt[slice_index,:,:])
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

interact(plot_slice, slice_index=IntSlider(min=0, max=pred.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [27]:
#show prediction overlaid on raw data
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(mark_boundaries_color(img[:,slice_index,:], pred[:,slice_index,:]))
    elif axis == 2:
        plt.imshow(mark_boundaries_color(img[:,:,slice_index], pred[:,:,slice_index]))
    else:
        plt.imshow(mark_boundaries_color(img[slice_index,:,:], pred[slice_index,:,:]))
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

interact(plot_slice, slice_index=IntSlider(min=0, max=pred.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [28]:
import scipy.ndimage
def label_foreground_structures(input_array, min_size=100000): #use aggressize min_size to remove small structures, then decent overlap stride to get ones that are erronously cut off on the edges
    # Find connected components in the foreground (value 2)
    foreground = (input_array == 2)
    
    # Label connected components
    labeled_array, num_features = scipy.ndimage.label(foreground)
    
    # Measure the size of each connected component
    component_sizes = np.bincount(labeled_array.ravel())
    
    # Create a mask for components larger than the minimum size
    large_components = component_sizes >= min_size
    
    # Ensure background is not considered a component
    large_components[0] = False
    
    # Create a filtered array to hold only large components
    filtered_array = labeled_array.copy()
    
    # Set small components to 0 (background)
    filtered_array[~large_components[labeled_array]] = 0
    
    print(f"Number of connected foreground structures before filtering: {num_features}")
    print(f"Number of connected foreground structures after filtering: {np.max(filtered_array)}")
    
    return filtered_array

In [29]:
labeled_arr = label_foreground_structures(pred)

Number of connected foreground structures before filtering: 56
Number of connected foreground structures after filtering: 6


In [30]:
#show post processing instance segmentation overlaid on raw data
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(mark_boundaries_color(img[:,slice_index,:], labeled_arr[:,slice_index,:]))
    elif axis == 2:
        plt.imshow(mark_boundaries_color(img[:,:,slice_index], labeled_arr[:,:,slice_index]))
    else:
        plt.imshow(mark_boundaries_color(img[slice_index,:,:], labeled_arr[slice_index,:,:]))
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

interact(plot_slice, slice_index=IntSlider(min=0, max=pred.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>